In [1]:
!pip install -q accelerate peft bitsandbytes transformers trl sentencepiece wandb huggingface_hub

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel, get_peft_model, TaskType
from trl import SFTTrainer, SFTConfig
from huggingface_hub import login
import wandb
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ['HF_HOME'] = r'E:\Users\Usuario\huggingface'
os.environ['HUGGINGFACE_HUB_CACHE'] = r'E:\Users\Usuario\huggingface\cache'
cache_dir =  r'E:\Users\Usuario\huggingface\cache'
hg_token = "***"
wandb_token = "***"
login(hg_token)
wandb.login(key=wandb_token)

dataset = load_dataset(
    "AndresCamargo/univalle_desarrollo_software_1",
    data_files="processed_dataset-qwen-alpaca.jsonl",
    split="train",
    revision="main"  
)

print(dataset[:5])

D:\Jupyter Notebooks\venvs\profevardilla\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Usuario\_netrc
wandb: Currently logged in as: andrescamargo (andrescamargo-universidad-del-valle) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
D:\Jupyter Notebooks\venvs\profevardilla\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` 

{'text': ['A continuación se presenta una instrucción que describe una tarea. Escribe una respuesta que complete adecuadamente la solicitud.\n\n### Instrucción:\n¿Cuál es tu profesión?\n\n### Respuesta:\nSoy un profesor.<|im_end|>', 'A continuación se presenta una instrucción que describe una tarea. Escribe una respuesta que complete adecuadamente la solicitud.\n\n### Instrucción:\n¿Dónde trabajas?\n\n### Respuesta:\nTrabajo en la Facultad de Ingeniería, específicamente en la Escuela de Ingeniería de Sistemas y Computación.<|im_end|>', 'A continuación se presenta una instrucción que describe una tarea. Escribe una respuesta que complete adecuadamente la solicitud.\n\n### Instrucción:\n¿Cuál es el año en el que tienes una conexión especial?\n\n### Respuesta:\nEl año 2025 tiene algún significado especial para mí, tal vez estoy relacionado con algún evento o programa que ocurre en ese año.<|im_end|>', 'A continuación se presenta una instrucción que describe una tarea. Escribe una respuest

In [6]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param_count = 0

    for param in model.parameters():
        num_params = param.numel()
        all_param_count += num_params
        if param.requires_grad:
            trainable_params += num_params

    print(f"Trainable parameters: {trainable_params}")
    print(f"All parameters: {all_param_count}")
    print(f"Trainable %: {100 * trainable_params / all_param_count:.4f}%")

In [ ]:
# Load base model
base_model_id = "Qwen/Qwen2.5-14B-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.float16,
    load_in_4bit=False,
    device_map={"": 0},
    cache_dir = cache_dir,
    resume_download=True
)
tokenizer = AutoTokenizer.from_pretrained(base_model_id, cache_dir=cache_dir)
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token
print(model)
print_trainable_parameters(model)

Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████| 8/8 [02:04<00:00, 15.58s/it]


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 5120)
    (layers): ModuleList(
      (0-47): 48 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=5120, out_features=5120, bias=True)
          (k_proj): Linear(in_features=5120, out_features=1024, bias=True)
          (v_proj): Linear(in_features=5120, out_features=1024, bias=True)
          (o_proj): Linear(in_features=5120, out_features=5120, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((5120,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((5120,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((5120,), eps=1e-06)
    (rotary_emb

In [4]:
# Load LoRA configuration
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj", "gate_proj", "down_proj", "up_proj"],
    lora_dropout=0.0,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, peft_config)

training_arguments = SFTConfig(
    output_dir="./output",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=16,
    save_total_limit=4,   
    logging_steps=4,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    report_to="wandb",
    run_name="Qwen2.5-14B-Instruct-desarrollo-software",
    push_to_hub=True,
    hub_model_id="Qwen2.5-14B-Instruct-DesarrolloSoftware1-Adapter",
    hub_strategy="checkpoint",
    packing=False,
)

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=dataset,
    processing_class=tokenizer
)

print("\n\n\npeft model: \n\n\n", model)
print(trainer.model.print_trainable_parameters())

Converting train dataset to ChatML:   0%|          | 0/1054 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/1054 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1054 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1054 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.





peft model: 


 PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 5120)
        (layers): ModuleList(
          (0-47): 48 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=5120, out_features=5120, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=5120, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
  

In [ ]:
trainer.train()

Step,Training Loss
4,2.304000
8,2.193100
12,1.636800
16,1.257200
20,0.933600
24,0.872200
28,0.872300
32,0.766800
36,0.685700
40,0.795400
